# GC Fisher Matrix Code

Authors: Alkistis Pourtsidou, Dida Markovic
    
Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

## Initialise parameters and import libraries

In [ ]:
# Set up science stuff
from __future__ import division
import numpy as np
import scipy, scipy.interpolate

# Set up all kinds of libraries
import os

# Import custom modules in this folder
import misc

# Importing CAMB
import camb

In [ ]:
# Options for running this notebook

### Paths
run_name = ''                 # Choose a name if you want, set to '' if don't want
outpath = './outputs/'        # Where to save outputs
TS = misc.get_timestamp()     # Arbitrary time stamp so linked output files are obvious
code_name = 'GC-Fish-'  

### Set up the precision of the interpolations
sig_figs = None               # If want to round the P(k) and kh
n_k_bin = 800                 # Settings for CAMB P(k)
minkh = 1e-4                  # Settings for CAMB P(k)
maxkh = 10.0                  # Settings for CAMB P(k)
interp_type = 'linear'        # Type of interpolation
k_per_logint = 200            # Setting for CAMB points per k decade

### Fiducial cosmological parameters
hubble_fid = 0.67
omegab_fid = 0.022445/hubble_fid**2
omegac_fid = 0.121203/hubble_fid**2
om0_fid = omegac_fid + omegab_fid
H00_fid = 100*hubble_fid
Ass_fid = 2.1265e-9
nss_fid = 0.96

# Dark Energy parameters
w0_fid = -1.0
wa_fid = 0.0
gamma_fid = 0.545

# Speed of light
c = 3.0e5

### Set up the survey, Euclid Definition Study Report (RedBook) 
### https://arxiv.org/abs/1110.3193
Area = 15000.0 #deg^2
omegatot = Area*pow(np.pi/180,2)
Dzbin = 0.1
sig_p = 0.001 

### Set up the redshift binned functions
zlist = np.arange(0.7,2.1,Dzbin)
Nzbins = len(zlist)
biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]
dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

### Set up the derivative steps
z_dep_step_default = 1e-5
shape_step = 1e-3

### Set up the Fisher Matrix integration (not power spectrum!)
kmin = 0.001
kmax = 0.2

### Set up the Fisher matrix calculation
params = {
    0: 'lnH',
    1: 'lnDA',
    2: 'lnfsig8',
    3: 'lnbsig8',
    4: 'Ps',
    5: 'ns',
    6: 'wb',
    7: 'wm',
    8: 'h'}
shape_params = [5,6,7,8]

## Define functions

In [ ]:
# Geometry functions for a spatially flat Universe

# Functions for DE
def w_integrand(z,w0=w0_fid,wa=wa_fid):
    return (1 + w0+wa*z/(z+1)) / (1+z)
def DE_evo(zc,w0=w0_fid,wa=wa_fid): 
    return np.exp(3*scipy.integrate.romberg(lambda z: w_integrand(z,w0,wa), 0, zc))

# Define E(z) = H(z)/H0
def Ez(zc,w0=w0_fid,wa=wa_fid):
    return np.sqrt((1-om0_fid)*DE_evo(zc,w0,wa) + om0_fid*pow(1+zc,3))
def Hz(zc,w0=w0_fid,wa=wa_fid):
    return Ez(zc,w0,wa)*H00_fid

# Define the cosmological distances
def drdz(zp,w0=w0_fid,wa=wa_fid):
    return (c/H00_fid)/Ez(zp,w0,wa)
def rcom(zc,w0=w0_fid,wa=wa_fid):
    return scipy.integrate.romberg(lambda z: drdz(z,w0,wa),0,zc)
def DA(zc,w0=w0_fid,wa=wa_fid):
    return rcom(zc,w0,wa)/(1+zc)

In [ ]:
# LCDM growth rate and growth factor

def fg(zz,w0=w0_fid,wa=wa_fid,gamma=gamma_fid):
    omz=om0_fid*pow(1+zz,3)/pow(Ez(zz,w0,wa),2)
    return pow(omz,gamma)

def Dg_dz(zz,w0=w0_fid,wa=wa_fid):
    return -fg(zz,w0,wa)/(1+zz)

def Dgz(zc,w0=w0_fid,wa=wa_fid):
    start_z = 0.0
    ans = scipy.integrate.romberg(lambda z: Dg_dz(z,w0,wa), start_z, zc)
    return np.exp(ans)